<a href="https://colab.research.google.com/github/RogerHeederer/NLP_entry/blob/master/TextGenerationLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AGABJG-u6m4pWwjPrvDuGsOb0EPSxLyLom2uRM1WIDeAuVIdHkJwtU
Mounted at /gdrive


In [5]:
import pandas as pd
df = pd.read_csv("/gdrive/My Drive/RogerHeederer/NLP_entry/data/Article2018.csv", low_memory=False)
df.head(3)

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...


In [6]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [7]:
print('열의 개수: ', len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [8]:
df['headline'].isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
1319    False
1320    False
1321    False
1322    False
1323    False
Name: headline, Length: 1324, dtype: bool

In [9]:
df['headline'].isnull().values.any()

False

In [10]:
headline = []
headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [11]:
#Unknown 노이즈 데이터 제거하기
print('총 샘플 개수 : {}'.format(len(headline)))

총 샘플 개수 : 1324


In [12]:
headline = [n for n in headline if n != 'Unknown']
print('노이즈값 제거 후 샘플 개수 : {}'.format(len(headline)))

노이즈값 제거 후 샘플 개수 : 1214


In [13]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [14]:
#데이터 전처리 하기
def repreprocessing(s):
  s = s.encode("utf8").decode("ascii", 'ignore')
  return ''.join(c for c in s if c not in punctuation).lower() # 구두점 제거 및 소문자화

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [15]:
#단어 집합 만들기
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기: %d' % vocab_size)

단어 집합의 크기: 3494


In [16]:
t.word_index['the'], t.word_index['a']

(1, 2)

In [17]:
#훈련 데이터 구성하기
sequences = list()

for line in text:
  encoded = t.texts_to_sequences([line])[0] # 각각 문장에 대한 정수 인코딩
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [18]:
# [[99, 269], # former nfl
#  [99, 269, 371], # former nfl cheerleaders
#  [99, 269, 371, 1115], # former nfl cheerleaders settlement
#  [99, 269, 371, 1115, 582], # former nfl cheerleaders settlement offer
#  [99, 269, 371, 1115, 582, 52], # 'former nfl cheerleaders settlement offer 1
#  [99, 269, 371, 1115, 582, 52, 7], # former nfl cheerleaders settlement offer 1 and
#  [99, 269, 371, 1115, 582, 52, 7, 2], # ... 이하 생략 ...
#  [99, 269, 371, 1115, 582, 52, 7, 2, 372],
#  [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
#  [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116], # 모든 단어가 사용된 완전한 첫번째 문장
#  # 바로 위의 줄 : former nfl cheerleaders settlement offer 1 and a meeting with goodell
#  [100, 3]] # epa to에 해당되며 두번째 문장이 시작됨.

In [19]:
#어떤 정수가 어떤 단어를 의미하는지 알아보자
index_to_word = {}
for key, value in t.word_index.items():
  index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [20]:
max_len=max(len(l) for l in sequences)
print('샘플 최대 길이 : {}'.format(max_len))

샘플 최대 길이 : 24


In [21]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [22]:
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]

In [23]:
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [24]:
print(y[:3])

[ 269  371 1115]


In [25]:
y = to_categorical(y, num_classes=vocab_size)

In [29]:
y.shape

(7803, 3494)

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 2s - loss: 7.6371 - accuracy: 0.0299
Epoch 2/200
244/244 - 1s - loss: 7.1160 - accuracy: 0.0287
Epoch 3/200
244/244 - 1s - loss: 6.9789 - accuracy: 0.0343
Epoch 4/200
244/244 - 1s - loss: 6.8527 - accuracy: 0.0417
Epoch 5/200
244/244 - 2s - loss: 6.7004 - accuracy: 0.0437
Epoch 6/200
244/244 - 1s - loss: 6.5335 - accuracy: 0.0477
Epoch 7/200
244/244 - 1s - loss: 6.3476 - accuracy: 0.0510
Epoch 8/200
244/244 - 1s - loss: 6.1547 - accuracy: 0.0590
Epoch 9/200
244/244 - 1s - loss: 5.9698 - accuracy: 0.0610
Epoch 10/200
244/244 - 1s - loss: 5.7948 - accuracy: 0.0656
Epoch 11/200
244/244 - 1s - loss: 5.6279 - accuracy: 0.0687
Epoch 12/200
244/244 - 1s - loss: 5.4715 - accuracy: 0.0729
Epoch 13/200
244/244 - 1s - loss: 5.3221 - accuracy: 0.0801
Epoch 14/200
244/244 - 1s - loss: 5.1776 - accuracy: 0.0832
Epoch 15/200
244/244 - 1s - loss: 5.0393 - accuracy: 0.0952
Epoch 16/200
244/244 - 1s - loss: 4.9062 - accuracy: 0.1047
Epoch 17/200
244/244 - 1s - loss: 4.7750 - accura

In [34]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [39]:
print(sentence_generation(model, t, 'i', 10))
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성

i want to be rich and im not sorry been hes


In [44]:
print(sentence_generation(model, t, 'former nfl', 10))

former nfl cheerleaders settlement offer 1 and a meeting with goodell and
